In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import preprocessing, model_selection, metrics
import lightgbm as lgb
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor

color = sns.color_palette()
%matplotlib inline

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 9999

lgbr = LGBMRegressor()

In [2]:
%run '../00_rh_settings.ipynb'
%run '../00_rh_load_processed.ipynb'

Populating the interactive namespace from numpy and matplotlib


NameError: name 'fix_nans' is not defined

In [3]:
train_df = df

In [45]:
import pandas as pd
import numpy as np

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

class EstimatorSelectionHelper:

    def __init__(self, models, params):
        if not set(models.keys()).issubset(set(params.keys())):
            missing_params = list(set(models.keys()) - set(params.keys()))
            raise ValueError("Some estimators are missing parameters: %s" % missing_params)
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}

    def fit(self, X, y, cv=3, n_jobs=3, verbose=1, scoring=None, refit=False):
        for key in self.keys:
            print("Running GridSearchCV for %s." % key)
            model = self.models[key]
            params = self.params[key]
            gs = RandomizedSearchCV(model, params, cv=cv, n_jobs=n_jobs,
                              verbose=verbose, scoring=scoring, refit=refit, n_iter=30,
                              return_train_score=True)
            gs.fit(X,y)
            self.grid_searches[key] = gs    

    def score_summary(self, sort_by='mean_score'):
        def row(key, scores, params):
            d = {
                 'estimator': key,
                 'min_score': min(scores),
                 'max_score': max(scores),
                 'mean_score': np.mean(scores),
                 'std_score': np.std(scores),
            }
            return pd.Series({**params,**d})

        rows = []
        for k in self.grid_searches:
            print(k)
            params = self.grid_searches[k].cv_results_['params']
            scores = []
            for i in range(self.grid_searches[k].cv):
                key = "split{}_test_score".format(i)
                r = self.grid_searches[k].cv_results_[key]        
                scores.append(r.reshape(len(params),1))

            all_scores = np.hstack(scores)
            for p, s in zip(params,all_scores):
                rows.append((row(k, s, p)))

        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)

        columns = ['estimator', 'min_score', 'mean_score', 'max_score', 'std_score']
        columns = columns + [c for c in df.columns if c not in columns]

        return df[columns]

In [46]:
from catboost import CatBoostClassifier

models1 = {
    'CatBoost': CatBoostClassifier(),}

params1 = {
    'CatBoost': {
        'iterations':[250,100,500,1000],
        'depth': range(2,10),
        'learning_rate': [0.001,0.05,0.1,0.2,0.3],
        'l2_leaf_reg':[3,1,5,10,100],
        'cat_features': [np.where(train_X.dtypes == np.object)[0]],
        'logging_level': ['Silent'],
    },
}

In [48]:
df.drop(columns=['Unnamed: 0',"loanKey","rep_loan_date","first_loan","first_overdue_date"], inplace=True, errors="ignore")

In [49]:
df.dropna(inplace=True)
df = df.replace(r'\s+', np.nan, regex=True)
df = df.replace('nan', np.nan)

In [51]:
train_X = df.drop(columns="bad_flag")
train_y = df["bad_flag"]

In [52]:
helper1 = EstimatorSelectionHelper(models1, params1)
helper1.fit(train_X, train_y, scoring='f1', n_jobs=-1)

Running GridSearchCV for CatBoost.
Fitting 3 folds for each of 30 candidates, totalling 90 fits


/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 17.6min
[Parallel(n_jobs=-1)]: Done  90 out of  90 | e

In [53]:
helper1.score_summary()

CatBoost


,estimator,min_score,mean_score,max_score,std_score,cat_features,depth,iterations,l2_leaf_reg,learning_rate,logging_level
0,CatBoost,0.207792,0.261495,0.326693,0.049217,"[4, 5]",9,250,5,0.2,Silent
21,CatBoost,0.201258,0.241173,0.273504,0.029978,"[4, 5]",4,500,1,0.3,Silent
19,CatBoost,0.191617,0.236863,0.304687,0.0488437,"[4, 5]",9,1000,3,0.3,Silent
25,CatBoost,0.153846,0.235751,0.302041,0.0614995,"[4, 5]",7,250,100,0.2,Silent
5,CatBoost,0.176871,0.234708,0.27027,0.0412553,"[4, 5]",2,1000,3,0.2,Silent
7,CatBoost,0.1625,0.227787,0.3083,0.0604887,"[4, 5]",7,500,10,0.3,Silent
24,CatBoost,0.190476,0.219446,0.246809,0.0230256,"[4, 5]",5,500,100,0.3,Silent
10,CatBoost,0.128571,0.213637,0.27234,0.0615826,"[4, 5]",2,1000,3,0.3,Silent
13,CatBoost,0.116788,0.210673,0.283186,0.0695921,"[4, 5]",7,250,10,0.1,Silent
17,CatBoost,0.142857,0.210436,0.251163,0.04812,"[4, 5]",7,250,3,0.1,Silent
